In [143]:
import matplotlib.pyplot as plt
import random


from matplotlib.colors import ListedColormap
from sklearn.datasets import make_classification, make_regression

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.metrics import accuracy_score

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

**Реализуем критерий информативности среднеквадратичного отклонения**

In [144]:
# сгенерируем данные
classification_data, classification_labels = make_classification(n_samples=1000, n_features=2, n_informative=2, 
                                                                 n_classes=2, n_redundant=0,
                                                                 n_clusters_per_class=1, random_state=5)

In [145]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле


In [146]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
            
        # найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)
        return prediction        

In [147]:
# Расчет критерия Джини

def gini(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 1
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2
        
    return impurity

In [148]:
# Расчет прироста

def gain(left_labels, right_labels, root_gini):

    # доля выборки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return root_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)

In [149]:
# Разбиение датасета в узле

def split(data, labels, column_index, t):
    
    left = np.where(data[:, column_index] <= t)
    right = np.where(data[:, column_index] > t)
        
    true_data = data[left]
    false_data = data[right]
    
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

## Домашнее задание

1. В коде из методички реализуйте один или несколько из критериев останова (количество листьев, количество используемых признаков, глубина дерева и т.д.)
**Внесла min_samples_leaf как аргумент который можно варьировать; <br>
Попыталась добавить параметр "Кол-во листьев". Получилось так себе, по-моему. Запуталась сама не понимаю где (см.ф-цию build_tree)**

In [150]:
# Нахождение наилучшего разбиения

def find_best_split(data, labels, min_samples_leaf):
    
    #  обозначим минимальное количество объектов в узле
#     min_samples_leaf = 3

    root_gini = gini(labels)

    best_gain = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique(data[:, index])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_samples_leaf or len(false_data) < min_samples_leaf:
                continue
            
            
            current_gain = gain(true_labels, false_labels, root_gini)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_gain > best_gain:
                best_gain, best_t, best_index = current_gain, t, index

    return best_gain, best_t, best_index

In [151]:
num_leafs = 0
# Построение дерева с помощью рекурсивной функции; по дефолту - 2 листа

def build_tree(data, labels, min_samples_leaf=1, max_num_leafs=np.inf, n_leafs=0, level=0, max_depth=np.inf):
    
    if max_num_leafs < 2:
        max_num_leafs = 2
     
    gain, t, index = find_best_split(data, labels, min_samples_leaf)
    
        
    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества ИЛИ
    # кол-во листьев превышает максимально допустимое ИЛИ глубина максимальная
    if gain == 0 or n_leafs+1 >= max_num_leafs or level== max_depth:
        return Leaf(data, labels)
    

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
    level+=1
    # Рекурсивно строим два поддерева
    n_leafs+=1
    true_branch = build_tree(true_data, true_labels, min_samples_leaf=min_samples_leaf, 
                             max_num_leafs=max_num_leafs, n_leafs=n_leafs, level=level, max_depth=max_depth)
    n_leafs+=1
    false_branch = build_tree(false_data, false_labels, min_samples_leaf=min_samples_leaf, 
                             max_num_leafs=max_num_leafs, n_leafs=n_leafs,level=level, max_depth=max_depth)
    
#   добавляем два листа и смотрим, можем ли строить false_branch    
#    если уже нельзя, возвращаем лист; иначе строим false_branch
#     if n_leafs >= max_num_leafs:
#         return Leaf(false_data, false_labels)
#     else:
#         n_leafs+=1
#         false_branch = build_tree(false_data, false_labels, min_samples_leaf=min_samples_leaf, 
#                              max_num_leafs=max_num_leafs, n_leafs=n_leafs)



    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [152]:
def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [153]:
def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [154]:
# Разобьем выборку на обучающую и тестовую

from sklearn.model_selection import train_test_split

train_data, test_data, train_labels, test_labels = train_test_split(classification_data, 
                                                                    classification_labels, 
                                                                    test_size=0.3,
                                                                    random_state=26)

In [155]:
# Построим дерево по обучающей выборке
my_tree = build_tree(train_data, train_labels, min_samples_leaf=3, max_num_leafs=10, max_depth = 4)

In [156]:
# Напечатаем ход нашего дерева
def print_tree(node, spacing=""):

    # Если лист, то выводим его прогноз
    if isinstance(node, Leaf):
        print(spacing + "Прогноз:", node.prediction)
        return

    # Выведем значение индекса и порога на этом узле
    print(spacing + 'Индекс', str(node.index), '<=', str(node.t))

    # Рекурсионный вызов функции на положительном поддереве
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Рекурсионный вызов функции на отрицательном поддереве
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")
    
print_tree(my_tree)

Индекс 0 <= -0.009400177353910522
--> True:
  Индекс 1 <= -1.3993975578815423
  --> True:
    Индекс 1 <= -1.6727874248522925
    --> True:
      Индекс 0 <= -1.1235801935520118
      --> True:
        Прогноз: 1
      --> False:
        Прогноз: 0
    --> False:
      Индекс 0 <= -0.7166853207483512
      --> True:
        Прогноз: 0
      --> False:
        Прогноз: 1
  --> False:
    Прогноз: 0
--> False:
  Индекс 1 <= -1.4358488277350174
  --> True:
    Прогноз: 0
  --> False:
    Индекс 0 <= 0.08948763365897316
    --> True:
      Индекс 0 <= 0.04459943514365716
      --> True:
        Прогноз: 1
      --> False:
        Прогноз: 1
    --> False:
      Индекс 1 <= -0.9251236805149236
      --> True:
        Прогноз: 1
      --> False:
        Прогноз: 1


In [157]:
# Получим ответы для обучающей выборки 
train_answers = predict(train_data, my_tree)

In [158]:
# И получим ответы для тестовой выборки
answers = predict(test_data, my_tree)

In [159]:
# Введем функцию подсчета точности как доли правильных ответов
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [160]:
# Точность на обучающей выборке
train_accuracy = accuracy_metric(train_labels, train_answers)
train_accuracy

96.71428571428572

In [161]:
# Точность на тестовой выборке
test_accuracy = accuracy_metric(test_labels, answers)
test_accuracy

95.33333333333334

2. *Реализуйте дерево для задачи регрессии. Возьмите за основу дерево, реализованное в методичке, заменив механизм предсказания в листе на взятие среднего значения по выборке, и критерий Джини на дисперсию значений.

In [162]:
def mse(array):
    mean = array.mean()
    return np.mean((array - mean)**2)

In [163]:
# сгенерируем данные
X, Y = make_regression(n_samples=1000, n_features=2, n_informative=2, n_targets=1, 
                                      noise=5, coef=False, random_state=2)

In [164]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле


Prediction теперь считаем просто как среднее в листе

In [165]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
            
        # найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = np.mean(self.labels)
        return prediction        

В приросте gini меняем на mse. MSE нужно минимизировать, значит прирост всё так же максимизировать

In [166]:
# Расчет прироста

def gain(left_labels, right_labels, root_mse):

    # доля выборки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return root_mse - p * mse(left_labels) - (1 - p) * mse(right_labels)

In [167]:
# Разбиение датасета в узле

def split(data, labels, column_index, t):
    
    left = np.where(data[:, column_index] <= t)
    right = np.where(data[:, column_index] > t)
        
    true_data = data[left]
    false_data = data[right]
    
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [168]:
# Нахождение наилучшего разбиения

def find_best_split(data, labels, min_samples_leaf):
    
    #  обозначим минимальное количество объектов в узле
#     min_samples_leaf = 30

    root_mse = mse(labels)

    best_gain = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique(data[:, index])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_samples_leaf or len(false_data) < min_samples_leaf:
                continue
            
            
            current_gain = gain(true_labels, false_labels, root_mse)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_gain > best_gain:
                best_gain, best_t, best_index = current_gain, t, index

    return best_gain, best_t, best_index

In [169]:
import time
# Построение дерева с помощью рекурсивной функции; по дефолту - 2 листа

def build_tree(data, labels, min_samples_leaf=1):
     
    gain, t, index = find_best_split(data, labels, min_samples_leaf)
    
        
    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if gain == 0:
        return Leaf(data, labels)
    
    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
   
    # Рекурсивно строим два поддерева

    true_branch = build_tree(true_data, true_labels, min_samples_leaf=min_samples_leaf)
    false_branch = build_tree(false_data, false_labels, min_samples_leaf=min_samples_leaf)


    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [170]:
def predict_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return predict_object(obj, node.true_branch)
    else:
        return predict_object(obj, node.false_branch)

In [171]:
def predict(data, tree):
    
    predictions = []
    for obj in data:
        prediction = predict_object(obj, tree)
        predictions.append(prediction)
    return predictions

In [172]:
# Разобьем выборку на обучающую и тестовую

from sklearn.model_selection import train_test_split

train_data, test_data, train_labels, test_labels = train_test_split(X, Y,
                                                                    test_size=0.3,
                                                                    random_state=26)

In [173]:
# Построим дерево по обучающей выборке
my_tree = build_tree(train_data, train_labels, min_samples_leaf=4)

In [174]:
# Напечатаем ход нашего дерева
def print_tree(node, spacing=""):

    # Если лист, то выводим его прогноз
    if isinstance(node, Leaf):
        print(spacing + "Прогноз:", node.prediction)
        return

    # Выведем значение индекса и порога на этом узле
    print(spacing + 'Индекс', str(node.index), '<=', str(node.t))

    # Рекурсионный вызов функции на положительном поддереве
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Рекурсионный вызов функции на отрицательном поддереве
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")
    
print_tree(my_tree)

Индекс 1 <= 0.2873168880922344
--> True:
  Индекс 0 <= 0.132924055186507
  --> True:
    Индекс 1 <= -1.1645267994987059
    --> True:
      Индекс 0 <= -1.1981403783170779
      --> True:
        Индекс 0 <= -2.2273412922974627
        --> True:
          Прогноз: -283.9515494035197
        --> False:
          Индекс 1 <= -1.5466746112644163
          --> True:
            Прогноз: -228.6007206595487
          --> False:
            Прогноз: -197.24143469809172
      --> False:
        Индекс 1 <= -1.7934355851948631
        --> True:
          Индекс 0 <= -0.4151419697731788
          --> True:
            Индекс 1 <= -2.1686184989032724
            --> True:
              Прогноз: -209.89766241295808
            --> False:
              Прогноз: -176.30139535209867
          --> False:
            Прогноз: -159.61504321699047
        --> False:
          Индекс 0 <= -0.5007914895154542
          --> True:
            Индекс 1 <= -1.4745636660271912
            --> True:
           

In [175]:
# Получим ответы для обучающей выборки 
train_answers = predict(train_data, my_tree)

In [176]:
# И получим ответы для тестовой выборки
answers = predict(test_data, my_tree)

Точность модели будем оценивать по MSE

In [177]:
# Введем функцию подсчета точности как доли правильных ответов
def mse_metric(actual, predicted):
    mse = sum((actual-predicted)**2)/len(actual)
    return mse

In [178]:
# Точность на обучающей выборке
train_mse = mse_metric(train_labels, train_answers)
train_mse

97.68866907901717

In [179]:
# Точность на обучающей выборке
test_mse = mse_metric(test_labels, answers)
test_mse

273.5851401280901

Достаточно переобученная модель получилось. Надо бы ввести ограничений